<a href="https://colab.research.google.com/github/kateMisis/misisexam/blob/main/%D0%9A%D1%80%D0%B8%D1%82%D0%B8%D1%87%D0%B5%D1%81%D0%BA%D0%B8%D0%B5_%D0%B3%D1%80%D1%83%D0%BF%D0%BF%D1%8B.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import csv
import networkx as nx
import itertools
from itertools import combinations
from itertools import groupby
import copy
import random
from google.colab import drive

population_URL = 'https://drive.google.com/uc?export=download&id=<1JNZ1jgEXVWxNreJU6HHA-QfPiTrihXpd>'
FAOSTAT_min_dietary_requirement_2014_2023_URL = 'https://drive.google.com/uc?export=download&id=<1JNZ1jgEXVWxNreJU6HHA-QfPiTrihXpd>'
food_supply_URL = 'https://drive.google.com/uc?export=download&id=<1JNZ1jgEXVWxNreJU6HHA-QfPiTrihXpd>'
trade_URL = 'https://drive.google.com/uc?export=download&id=<1JNZ1jgEXVWxNreJU6HHA-QfPiTrihXpd>'


population = '/content/drive/My Drive/population_2023_projected.csv'
FAOSTAT_min_dietary_requirement_2014_2023 = '/content/drive/My Drive/FAOSTAT_min_dietary_requirement_2014_2023-2.csv'
food_supply = '/content/drive/My Drive/food_supply.csv'
trade = '/content/drive/My Drive/trade_all_w1.csv'


pop = dict()
pop_file = open(population, 'r') #Reading csv file
pop_readlines = pop_file.readlines() #Read the file line by line
for k in pop_readlines: #Go through the lines we've read
    c = k.split(',') #Separate these lines by the separator ","
    if k != pop_readlines[0]:
      pop[c[2]] = int(c[15])

FAO_countries = set()

FAO_min_data = dict() #A data array network_data is created, which will store the edges of our network (vertex1, vertex2, weight)
FAO_min_file = open(FAOSTAT_min_dietary_requirement_2014_2023, 'r') #Reading csv file
FAO_min_readlines = FAO_min_file.readlines() #Read the file line by line
for k in FAO_min_readlines: #Go through the lines we've read
    c = k.split(',') #Separate these lines by the separator ","
    if (k != FAO_min_readlines[0]) and (c[8] == '2023'):
        FAO_countries.add(c[3])
        FAO_min_data[c[3]] = float(c[11])

quota = dict()
q_data = [] #A data array network_data is created, which will store the edges of our network (vertex1, vertex2, weight)
q_file = open(food_supply, 'r') #Reading csv file
q_readlines = q_file.readlines() #Read the file line by line
for k in q_readlines: #Go through the lines we've read
    c = k.split(',') #Separate these lines by the separator ","
    if k != q_readlines[0]:
        if c[3] in FAO_countries:
            quota[c[3]] = float(((float(c[19])-FAO_min_data[c[3]])*pop[c[3]])*365) #Write data on the quotas of the considered vertices into the dictionary
            q_data.append((c[3], ((float(c[19])-FAO_min_data[c[3]])*pop[c[3]])*365)) #Write the data of the line edge through "," into a tuple and write it into the network_data array

n_data = [] #A data array network_data is created, which will store the edges of our network (vertex1, vertex2, weight)
n_file = open(trade, 'r') #Reading csv file
n_readlines = n_file.readlines() #Read the file line by line
for k in n_readlines: #Go through the lines we've read
    c = k.split(',') #Separate these lines by the separator ","
    if (k != n_readlines[0]) and (c[0] == '2023'):
        if (c[1] in FAO_countries) and (c[2] in FAO_countries):
            n_data.append((c[1], c[2], float(c[3]))) #Write the data of the line edge through "," into a tuple and write it into the network_data array

# Network creation
Network = nx.DiGraph() #An empty graph is created for further creation of a directed graph.
Network.add_weighted_edges_from(n_data) # Directed edges with weights are added to the empty graph

Сountry = input('Введите название страны, чьи критические группы Вы желаете узнать: ')

k_value = 3

def In_degree (G):
  In_degree = dict() #A dictionary In_degree is created, which will store for each vertex its In-degree value.
  for j in nx.nodes(G): #We go through the list of vertices of a given network G
      in_degree = 0 #For each vertex, a variable in_degree is created, which is initially equal to 0
      for n in G.predecessors(j): #For a given vertex j, we iterate through the list of vertices from which edges go to j
        in_degree += float(G[n][j]['weight']) #For each such vertex n, we add to the variable in_degree the weight of the edge from n to j
      In_degree[j] = in_degree #We write the value of the variable In_degree for the vertex j into the dictionary In_degree
  return In_degree

Crtitcal_groups = [] #For each vertex j, it is created a list that will contain the critical groups for vertex j
for k in range(min(k_value, Network.in_degree(Сountry))): #Set the number of vertices that can form a critical group
  if In_degree(Network)[Сountry] != 0 and (Сountry in quota): #Test for zero centrality In_degree
    for elem in combinations(Network.predecessors(Сountry), k+1): #Go through all possible combinations of elem from k+1 vertices from which an edge goes to vertex j
      loc_nodes = [] #For each vertex, we create an array loc_nodes, which will contain the vertices from the critical group under consideration.
      loc_nodes.append(Сountry) #Add node j to the loc_nodes array
      for l in elem: #We go through all possible combinations of k+1 vertices
        loc_nodes.append(l) #Add each combination to the loc_nodes array
        G_lokal = nx.subgraph(Network, loc_nodes) #Create a subgraph G_lokal of a given network G on the nodes from the loc_nodes array
        if In_degree(G_lokal)[Сountry] >= quota[Сountry]: #We set a check whether the In-degree for the considered vertex j in the subgraph G_lokal is greater than the given quota for the vertex j
          Crtitcal_groups.append(elem) # In the array Crtitcal_groups we add the corresponding combination of elem
print(Сountry, 'заивисит от следующих критических групп: ')
for k in Crtitcal_groups:
  print(Сountry, 'зависит от группы состоящей из ', k[0], ',', k[1], 'и', k[2])

FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/My Drive/population_2023_projected.csv'

In [ ]:
print(Crtitcal_groups)